## Optimal Bidding

Now we will use the posterior distribution to compute the optimal bid, defined as the bid that maximizes expect return.

First we present the methods top-down (showing how they are used, before how they work).

To compute optimal bids, the class called `GainCalculator` was created:

In [1]:
class GainCalculator(object):
    """Encapsulates computation of expected gain."""
    def __init__(self, player, opponent):
        self.player = player
        self.opponent = opponent

`player` and `opponent` are `Player` objects.

`GainCalculator` provides `ExpectedGains`, which computes a sequence of bids and expected gain for each bid:

In [2]:
def ExpectedGains(self, low=0, high=75000, n=101):
    """Computes expected gains for a range of bids.
        low: low bid
        high: high bid
        n: number of bids to evaluates
        returns: tuple (sequence of bids, sequence of gains)
    
        """
    bids = numpy.linspace(low, high, n) # a list of bids going up in 101
    
    gains = [self.ExpectedGain(bid) for bid in bids] # ExpectedGain is defined below
    
    return bids, gains

`low` and `high` specify the range of possible bids; `n` is the number of bids to try.

`ExpectedGains` calls `ExpectedGain`, which computes expected gain for a given bid:

In [3]:
def ExpectedGain(self, bid):
    """Computes the expected return of a given bid.
        bid: your bid
        """
    suite = self.player.posterior # this was created in def MakeBeliefs from self.prior = Price(pmf, self)
    total = 0
    for price, prob in sorted(suite.Items()): 
        gain = self.Gain(bid, price)
        total += prob * gain
    return total

`ExpectedGain` loops through the values in the posterior and computers the gain for each bid, given the actual prices of the showcase. It weights each gain with the corresponding probability and returns the total.

`ExpectedGain` invokes `Gain`, which takes a bid and an actual price and returns the expected gain:

In [5]:
def Gain(self, bid, price):
    if bid > price:
        return 0
    diff = price - bid
    prob = self.ProbWin(diff)
    
    if diff <= 250:
        return 2 * price * prob
    else:
        return price * prob

If you overbid, you get nothing. Otherwise, we computer the difference between your bid nad the price, which determines your probability of winning.

If `diff` is less than $250, you win both showcases. 

Finally, we have to compute the probability of winning based on diff:

In [6]:
def ProbWin(self, diff):
    prob = (self.opponent.ProbOverbid() + self.opponent.ProbWorseThan(diff))
    return prob

If your opponent overbids, you win. Otherwise, you have to hope that your opponent is off by more than `diff`. `Player` provides methods to compute both probabilities:

In [8]:
# class Player
   # def ProbOverbid(self):
    #    return self.cdf_diff.Prob(-1)
    
    #def ProbWorseThan(self, diff):
     #   return 1 - self.cdf_diff.Prob(diff)

This code is from the view fo the opponent: "What is the probability that I overbid?"  and "What is the probability that my bid is off by more than `diff`"

Both answers are based on the CDF of `diff`. If the opponents `diff` is less than or equal, you win. If the opponent's `diff` is worse than yours, you win. Otherwise you lose.

Finally, here is the code that computes the optimal bids:

In [10]:
# class Player
    def OptimalBid(self, guess, opponent):
        self.MakeBeliefs(guess)
        calc = GainCalculator(self, opponent)
        bids, gains = calc.ExpectedGains()
        gain, bid = max(zip(gains, bids))
        return bid, gain

Given a guess and an opponent, `OptimalBid` computes the posterior distribution, instantiates a `GainCalculator`, computes expected gains for a range of bids and returns the optimal bid and expected gain, Whew!

Figure 6.4 shows the results for both players, based on a scenario where Player 1's best guess is \$20000 and Player 2's best guess is \$40000

<img src="thinkbayesprice3.png">